In [1]:
import pandas as pd
import spacy
import nltk
import json
from rake_nltk import Rake
import os
import re
import pytextrank

C:\Users\Kiki\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## 1. RAKE

In [2]:
def rake(sentences):
    keyphrases = dict()
    r = Rake()
    for index, row in sentences.iterrows():
        r.extract_keywords_from_text(row['text'])
        if row['id'] not in keyphrases.keys():
            keyphrases[row['id']] = list()
        keyphrases[row['id']].append(r.get_ranked_phrases())
    return keyphrases

In [3]:
os.chdir('D:/TU_Graz/Thesis/Datasets/Reddit_preprocessed')
submissions = pd.read_csv("train_submissions_simplified.csv")

# Remove punctuations
subs = submissions[['id', 'text']]
for i in range(0, len(subs['text'])):
    tmp = subs.iloc[i]['text']
    subs.iloc[i]['text'] = re.sub(r'[^\w\s]', '', tmp)
    tmp = subs.iloc[i]['text'] 
    subs.iloc[i]['text'] = re.sub('NEW_LINE', '', tmp)
    tmp = subs.iloc[i]['text'] 
    subs.iloc[i]['text'] = re.sub('new_line', '', tmp)

In [4]:
keyphrases = rake(subs)
with open("D:/TU_Graz/Thesis/Datasets/rake_keywords_format_1_TRAIN.json", "w") as outfile:
    json.dump(keyphrases, outfile)

## 2. TextRank

In [5]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank")

def textrank_f(sentences):
    keyphrases = dict()
    for index, row in sentences.iterrows():
        doc = nlp(row['text'])
        #if row['id'] not in keyphrases.keys():
        #    keyphrases[row['id']] = list()
        phrases = [phrase.text for phrase in doc._.phrases]
        keyphrases[row['id']] = phrases
    return keyphrases

In [6]:
keyphrases = textrank_f(subs)

In [7]:
with open("D:/TU_Graz/Thesis/Datasets/textrank_keywords_format_1_TRAIN.json", "w") as outfile:
    json.dump(keyphrases, outfile)